In [1]:
import torch 
from torch import nn
import torchvision
from torchvision import transforms
from torchvision.transforms import ToTensor
import torchmetrics

In [ ]:
from torch.utils.data import DataLoader



In [ ]:
class SARDespeckleModel(nn.Module):
    def __init__(self,):
        super().__init__()
        self.layer_1 = nn.Sequential(
            nn.Conv2d()
        )